# Математический анализ - 1, ФКН ВШЭ

## Лабораторная работа 1: Методы интерполяции функции

### Оценивание и штрафы
Всего заданий: **12**, максимальная оценка — **12 баллов (0.1 бонус)**. После мягкого дедлайна оценка снижается на 1 балл. После жесткого дедлайна работы не принимаются.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом пользоваться запрещается,даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке (например, лишние циклы, `np.vectorize`, `np.apply_along_axis` и так далее). Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

Использование генеративных языковых моделей разрешено только в случае явного указания на это. Необходимо прописать (в соответствующих пунктах, где использовались, либо в начале/конце работы):

- какая языковая модель использовалась
- какие использовались промпты и в каких частях работы
- с какими сложностями вы столкнулись при использовании генеративных моделей, с чем они помогли больше всего
  
Copy-paste ответа генеративной модели запрещается (кроме графиков, но все равно нужно явно прописывать использование)

**Мягкий дедлайн: 17.02.2026 23:59 (по МСК)**

**Жесткий дедлайн: 20.02.2026 23:59 (по МСК)**

**Сдавать сюда: [Классрум](https://classroom.google.com/c/ODA1OTA4OTM5NzMy/a/ODE4ODc5NjQ1MDM0/details)**

### О задании
В данной лабораторной работе вы познакомитесь с методами интерполяции и аппроксимации функций.
Вам предстоит реализовать классические методы (Лагранж, Ньютон), изучить проблему сходимости интерполяционных процессов (феномен Рунге) и способы её решения (узлы Чебышева), а также поработать с более продвинутыми методами (сплайны, RBF) и многомерной интерполяцией

P.S Пожалуйста, аккуратно оформляйте графики, ориентироваться можно на [это](https://github.com/esokolov/ml-course-hse/blob/master/2022-fall/seminars/sem02-charts.ipynb). У графиков обязательно должно быть:

- Название
- Подписанные оси
- Легенда, если необходимо (например, если несколько графиков на одном рисунке)
- Все должно быть четко видно и ничего не сливаться
- За некрасивые графики можем снизить баллы

In [6]:
from typing import Callable, Literal

from PIL import Image
import torch
import torch.nn.functional as F
from torchvision.transforms import v2
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

## Аппроксимация функции

На практике часто необходимо приблизить сложную функцию в более простом виде, но без существенной потери точности.

### Многочлен Тейлора

Начнем с самого простого, но не самого эффективного способа.

Пусть функция $f(x)$ бесконечно раз дифференцируема. Как мы знаем, тогда мы можем в окрестности точки $x_0$ построить ряд Тейлора, чтобы ее аппроксимировать:

$$
f(x) = f(x_0) + f'(x_0)(x - x_0) + \frac{f''(x_0)}{2!}(x - x_0)^2 + \ldots + \frac{f^{(n)}(x_0)}{n!}(x - x_0)^n + \ldots
$$

Мы будем аппроксимировать ее конечным числом слагаемых:

$$
f(x) \approx f(x_0) + f'(x_0)(x - x_0) + \frac{f''(x_0)}{2!}(x - x_0)^2 + \ldots + \frac{f^{(n)}(x_0)}{n!}(x - x_0)^n + \frac{f^{(n+1)}(c)}{n!} (x - x_0)^{n+1}, c \in (x_0, x)
$$

**Задание 0 (0.5 баллов):** Постройте многочлен Тейлора $p_n(x)$ в точке $x_0 = 0$ на отрезке $[-1, 1]$ для функции $f(x) = \frac{1}{1 + 25x^2}$. Постройте график функции и многочлена Тейлора при разных $n$. Что вы можете сказать о качестве аппроксимации?

**Ответ:**

In [ ]:
def f(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + 25 * x**2)

def taylor_poly(x: np.ndarray, n: int) -> np.ndarray:
    # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
    raise NotImplementedError

## Классическая интерполяция

Понятно, что многочлен Тейлора не самый эффективный способ приблизить функцию, так как требует большого количества слагаемых (соответственно, много вычислений производных), чтобы хорошо аппроксимировать ее, особенно если нужно приблизить не в окрестности конкретной точки, а на интервале.

Вместо этого можно использовать другой подход — интерполяцию.

In [ ]:
def f1(x: np.ndarray) -> np.ndarray:
    return x**2

def df1dx(x: np.ndarray) -> np.ndarray:
    return 2*x

def f2(x: np.ndarray) -> np.ndarray:
    return np.sin(x)

def df2dx(x: np.ndarray) -> np.ndarray:
    return np.cos(x)

def test_interpolation(
    func: Callable[[np.ndarray], np.ndarray],
    interpolator_cls,
) -> None:
    nodes = np.linspace(-1, 1, 20)
    values = func(nodes)

    interpolator = interpolator_cls(nodes, values)

    x_test = np.random.default_rng(0).uniform(-1, 1, 50)
    predicted = interpolator.predict(x_test)
    target = func(x_test)

    assert np.allclose(predicted, target), (
        f"{interpolator_cls.__name__} interpolation failed!"
    )
    print(f"{interpolator_cls.__name__} check passed")

### Многочлен Лагранжа

Пусть заданы $n + 1$ узлов интерполяции $(x_0, y_0), (x_1, y_1), \ldots (x_n, y_n)$, где $x_i$ — разные точки, $y_i$ — значение искомой функции $f(x)$

Многочлен Лагранжа — многочлен $L_n(x)$ степени не выше $n$ такой, что $L_n(x_i) = y_i$, $i = 0, 1, \ldots, n$. Он задается следующим образом:

$$
L_n(x) = \sum\limits_{i = 0}^n y_i l_i(x),
$$

где $l_i(x)$ — базисные многочлены Лагранжа такой, что

$$
l_i(x) = \begin{cases}
1, x = x_i\\
0, x \neq x_i
\end{cases}
$$

$$
l_i(x) = \prod\limits_{j = 0, j \neq i} \frac{x - x_j}{x_i - x_j}
$$

**Задание 1 (0.5 баллов):** Реализуйте `LagrangePolynomial`

In [ ]:
class LagrangePolynomial:
    def __init__(self, nodes: np.ndarray, values: np.ndarray) -> None:
        """
        Initialize the Lagrange Polynomial interpolator.

        Parameters
        ----------
        nodes : np.ndarray
            The x-coordinates of the interpolation nodes.
        values : np.ndarray
            The y-coordinates (function values) corresponding to the nodes.
        """
        self.nodes = nodes.copy()
        self.values = values.copy()

    def compute_lagrange_basis(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the Lagrange basis polynomials evaluated at x.

        Parameters
        ----------
        x : np.ndarray
            The points at which to evaluate the Lagrange basis polynomials.

        Returns
        -------
        np.ndarray
            A 2D array of shape (len(x), len(nodes)) where each column represents
            a Lagrange basis polynomial evaluated at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict the interpolated values at x using the Lagrange Polynomial.

        Parameters
        ----------
        x : np.ndarray
            The points at which to interpolate the function.

        Returns
        -------
        np.ndarray
            The interpolated values at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

test_interpolation(f1, LagrangePolynomial)
test_interpolation(f2, LagrangePolynomial)

### Многочлен Ньютона

Как известно, у многочлена Лагранжа есть несколько недостатков:
- требуется постоянно пересчитывать базисные многочлены
- если добавится еще один узел, то нужно заново все пересчитывать

Поэтому перейдем к более эффективному методу — к многочлену Ньютона.

Интерполяционный многочлен в форме Ньютона выглядит следующим обраозм:
$$
p(x) = f[x_0] + f[x_0, x_1] (x - x_0) + ... + f[x_0, ... , x_n] (x - x_0) ... (x - x_{n-1})
$$

$f[x_i, ... , x_j]$ - разделенные разности, которые могут быть вычислены рекурсивно:
$$
f[x_i] = y_i
$$
$y_i$ - значение функции в узлах интерполяции
$$
f[x_i, ..., x_j] = \frac{f[x_{i + 1}, ..., x_j] - f[x_i, ..., x_{j - 1}]}{x_j - x_i}
$$

**Задание 2 (0.5 баллов):** Реализуйте `NewtonPolynomial`

In [ ]:
class NewtonPolynomial:
    def __init__(self, nodes: np.ndarray, values: np.ndarray):
        """
        Initialize the Newton Polynomial interpolator.

        Parameters
        ----------
        nodes : np.ndarray
            The x-coordinates of the interpolation nodes.
        values : np.ndarray
            The y-coordinates (function values) corresponding to the nodes.
        """
        self.nodes = nodes
        self.values = values
        self.coeffs = self._divided_differences()

    def _divided_differences(self) -> np.ndarray:
        """
        Compute the coefficients of the Newton interpolating polynomial
        using divided differences.

        Returns
        -------
        np.ndarray
            An array containing the coefficients of the Newton polynomial.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict the interpolated values at x using the Newton Polynomial.

        Parameters
        ----------
        x : np.ndarray
            The points at which to interpolate the function.

        Returns
        -------
        np.ndarray
            The interpolated values at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

test_interpolation(f1, NewtonPolynomial)
test_interpolation(f2, NewtonPolynomial)

**Задание 3 (0.5 балла):**

- Постройте график $f(x)$ (возьмите ту же функцию из многочлена Тейлора) и $p_n(x)$ на отрезке $[-5, 5]$ (для Лагранжа и Ньютона отдельно)
- Посчитайте ошибку аппроксимации $r(n)$ на тестовых точках $\xi_1, \ldots, \xi_N$:

$$
r(n) = \frac{1}{N}\sum_{i = 1}^N|f(\xi_i) - p_n(x_i)|
$$

Проделайте пункты выше для всех $n \in \{3, 5, 10, 15, 20, 25, 50\}$ и постройте график зависимости $r(n)$ (в логарифмической шкале $\log_{10} n$ vs $log_{10} r(n)$. Сравните $r(n)$ при равномерной генерации узлов и рандомной

**Вопрос:** Как ведут себя графики ошибки с ростом $n$? Какой многочлен лучше оказался? Как ведут себя многочлены на концах отрезка?

**Ответ:**

In [ ]:
# Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*

### Барицентрическая форма многочлена Лагранжа

У классической формы многочлена Лагранжа есть ещё один практический недостаток: прямое вычисление

$$
L_n(x) = \sum\limits_{i = 0}^n y_i l_i(x),
$$

часто получается численно неустойчивым (особенно при больших
n
n и “плохих” узлах), а также требует много умножений/делений при каждом новом $x$

Определим барицентрические веса:
$$
w_j=\frac{1}{\prod_{k\ne j}(x_j-x_k)}
$$

Также введём вспомогательный многочлен $\omega(x)=\prod_{k=0}^n (x-x_k)$

Тогда интерполяционный многочлен можно записать как
$$
p(x)=\omega(x)\sum_{j=0}^n \frac{w_j y_j}{x-x_j}.
$$

На практике чаще используют эквивалентную вторую барицентрическую формулу, более устойчивую форму:
$$
p(x)=\frac{\sum_{j=0}^n \dfrac{w_j y_j}{x-x_j}}{\sum_{j=0}^n \dfrac{w_j}{x-x_j}}
$$

**Задание 4 (0.5 баллов):** Реализуйте `BarycentricLagrangePolynomial`, который:

- по узлам $x_j$ предварительно вычисляет веса $w_j$
- по запросу значения в точке $x$ вычисляет $p(x)$ по формуле:

    $$
    p(x)=\frac{\sum_{j=0}^n \dfrac{w_j y_j}{x-x_j}}{\sum_{j=0}^n \dfrac{w_j}{x-x_j}}
    $$

и повторите задание 2, сравнив с классическими многочленами. Получилось ли улучшить интерполяцию?


**Ответ:**


In [ ]:
class BarycentricLagrangePolynomial:
    def __init__(self, nodes: np.ndarray, values: np.ndarray) -> None:
        """
        Initialize the Barycentric Lagrange Polynomial interpolator.

        Parameters
        ----------
        nodes : np.ndarray
            The x-coordinates of the interpolation nodes.
        values : np.ndarray
            The y-coordinates (function values) corresponding to the nodes.
        """
        self.nodes = nodes.copy()
        self.values = values.copy()
        self.weights = ... # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict the interpolated values at x using the Barycentric Lagrange Polynomial.

        Parameters
        ----------
        x : np.ndarray
            The points at which to interpolate the function.

        Returns
        -------
        np.ndarray
            The interpolated values at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

test_interpolation(f1, BarycentricLagrangePolynomial)
test_interpolation(f2, BarycentricLagrangePolynomial)

**Задание 5 (0.5 баллов):** Возьмем теперь другие узлы $x_i$:

$$
x_i = \cos\left(\frac{2k + 1}{2(n + 1)}\pi\right), \quad k = 0, \ldots, n
$$


Сравните все многочлены с такой генерацией узлов. Чтобы перенести узлы на другой отрезок, воспользуйтесь линейным масштабированием. Получилось ли улучшить интерполяцию? Почему получилось / не получилось?

In [ ]:
# Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*

## Эрмитова интерполяция

Часто на практике нам необходимо, чтобы интерполяционный многочлен не только хорошо аппроксимировал значения функции, но и учитывал ее гладкость — так называемая [эрмитова интерполяция](https://ru.wikipedia.org/wiki/%D0%AD%D1%80%D0%BC%D0%B8%D1%82%D0%BE%D0%B2%D0%B0_%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BF%D0%BE%D0%BB%D1%8F%D1%86%D0%B8%D1%8F)

### Многочлен Лагранжа

Вернемся к многочлену Лагранжа и адаптируем его для эрмитовой интерполяции следующим образом: введем многочлены $A_i(x)$ и $B_i(x)$:

$$
A_i(x_j) =
\begin{cases}
    1, i = j\\
    0, i \neq j
\end{cases}
\,
A'_i(x_j) = 0
$$

$$
B'_i(x_j) =
\begin{cases}
    1, i = j\\
    0, i \neq j
\end{cases}
\,
B_i(x_j) = 0
$$

В таком случае интерполяционный многочлен строится следующим образом:

$$
p(x) = \sum\limits_{i = 1}^n y_i A_i(x) + \sum\limits_{i = 1}^n y'_i B_i(x)
$$

Многочлены $A_i(x)$ и $B_i(x)$ определим следующим образом:

$$
\begin{cases}
A_i(x) = (1 - 2(x - x_i) \cdot l'_i(x_i)) \cdot l_i^2(x)\\
B_i(x) = (x - x_i) \cdot l_i^2(x)
\end{cases}
$$

Здесь $l_i(x)$ - базисные многочлены Лагранжа

**Задание 6 (1 балл):** Проверьте, что многочлены $A$ и $B$ удовлетворяют условиями. Найдите значения производной базисного многочлена $l_i(x_i)$ аналитически. Реализуйте класс `HermiteLagrangePolynomial`:

**Ответ:**

In [ ]:
class HermiteLagrangePolynomial:
    def __init__(self, nodes: np.ndarray, values: np.ndarray, der_values: np.ndarray) -> None:
        """
        Initialize the Hermite Lagrange Polynomial with derivative values.

        Parameters
        ----------
        nodes : np.ndarray
            The x-coordinates of the interpolation nodes.
        values : np.ndarray
            The y-coordinates (function values) corresponding to the nodes.
        der_values : np.ndarray
            The first derivatives of the function at the nodes.
        """
        self.nodes = nodes
        self.values = values
        self.der_values = der_values

    def compute_lagrange_basis(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the Lagrange basis polynomials evaluated at x.

        Parameters
        ----------
        x : np.ndarray
            The points at which to evaluate the Lagrange basis polynomials.

        Returns
        -------
        np.ndarray
            A 2D array of shape (len(x), len(nodes)) where each column represents
            a Lagrange basis polynomial evaluated at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

    def compute_A_B_polynomials(self, x: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
                """
        Compute the A(x) and B(x) polynomials for the Hermite interpolation.

        Parameters
        ----------
        x : np.ndarray
            The points at which to compute A(x) and B(x).

        Returns
        -------
        tuple[np.ndarray, np.ndarray]
            - A(x): 2D array of the Lagrange polynomials weighted by derivatives.
            - B(x): 2D array of the standard Lagrange polynomials.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict the interpolated values at x using the Hermite Lagrange Polynomial.

        Parameters
        ----------
        x : np.ndarray
            The points at which to interpolate the function.

        Returns
        -------
        np.ndarray
            The interpolated values at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

### Многочлен Ньютона

К сожалению, многочлен Лагранжа сложно адаптируется под случаи, когда заданы производные старших степеней, поэтому чаще используют многочлены Ньютона с разделенныи разностями с повторениями.

Интерполяционный многочлен в форме Ньютона выглядит следующим образом:
$$
p(x) = f[x_0] + f[x_0, x_1] (x - x_0) + ... + f[x_0, ... , x_n] (x - x_0) ... (x - x_{n-1})
$$
$f[x_i, ... , x_j]$ - разделенные разности, которые могут быть вычислены рекурсивно:
$$
f[x_i] = y_i
$$
$y_i$ - значение функции в узлах интерполяции
Если $x_i \neq x_j$:
$$
f[x_i, ..., x_j] = \frac{f[x_{i + 1}, ..., x_j] - f[x_i, ..., x_{j - 1}]}{x_j - x_i}
$$
В противном случае:
$$
f[x_i, ..., x_j] = \frac{1}{(j - i)!} \frac{\partial^{j - i} f(x_i)}{\partial x ^{j - i}} (производная \, порядка \, j - i)
$$

Теперь у нас узел $x_i$ будет повторяться $k+1$ раз, так как мы знаем не только значение $f(x_i$), но и $k$ производных $f^{(j)}(x_i)$

**Задание 7 (1 балл)**: Реализуйте `HermiteNewtonPolynomial`. Узлы должны быть отсортированы по возрастанию

In [ ]:
def generate_interpolation_nodes_with_rep(nodes: np.array, max_der_order: int)-> np.ndarray:
    return np.sort(
        np.repeat(
            np.linspace(segment[0], segment[1], num_nodes),
            repeats=max_der_order + 1,
        )
    )


class HermiteNewtonPolynomial:
    def __init__(self, nodes: np.ndarray, values: np.ndarray) -> None:
        """
        Initialize the Hermite Newton Polynomial interpolator.

        Parameters
        ----------
        nodes : np.ndarray
            The x-coordinates of the interpolation nodes.
        values : np.ndarray
            2D array of shape (n, k+1) containing the data to be interpolated.
            The entry values[i, j] stores the j-th derivative of the function at
            nodes[i], i.e.:
                values[i, 0] = f(nodes[i]),
                values[i, 1] = f'(nodes[i]),
                ...
                values[i, k] = f^(k)(nodes[i]).
            Here n is the number of nodes, and k is the maximum derivative order
            enforced by the Hermite conditions.
        """
        max_der_order = values.shape[1] - 1
        self.nodes = generate_interpolation_nodes_with_rep(nodes, max_der_order)
        self.values = values
        self.coeffs = self._divided_differences()

    def _divided_differences(self) -> np.ndarray:
        """
        Compute the coefficients of the Hermite Newton interpolating polynomial
        using divided differences.

        Returns
        -------
        np.ndarray
            An array containing the coefficients of the Newton polynomial.
        """
        pass

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict the interpolated values at x using the Hermite Newton Polynomial.

        Parameters
        ----------
        x : np.ndarray
            The points at which to interpolate the function.

        Returns
        -------
        np.ndarray
            The interpolated values at x.

        Notes
        -----
        This implementation does not use loops over `x`. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*
        raise NotImplementedError

**Задание 8 (0 баллов):** Баллы за задания 6 и 7 зачтутся при выполнении этого. Вам снова нужно сравнить качество интерполяции с классическими многочленами (можно не всеми, но лучшими). Проверьте так же, что условия на равенство производных так же выполняются.

In [ ]:
# Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*

## Интерполяция сплайнами

Кубические сплайны — это кусочные многочлены третьей степени, которые используются для интерполяции данных и создания гладких кривых. Они обеспечивают непрерывность и гладкость между узлами, что делает их популярным выбором в численных методах, компьютерной графике и приложениях, связанных с обработкой данных

### Основные характеристики кубических сплайнов:

1. **Форма полинома**:
   Кубический полином на интервале $[x_i, x_{i+1}]$ имеет вид:
   $$
   S_i(x) = a_i + b_i (x - x_i) + c_i (x - x_i)^2 + d_i (x - x_i)^3
   $$
   где $a_i, b_i, c_i, d_i$ — коэффициенты, которые нужно определить

2. **Условия непрерывности**:
   Чтобы сплайн был гладким, необходимо удовлетворять определённым условиям:
   - **Непрерывность**: Значения сплайнов в узловых точках должны совпадать:
     $$
     S_i(x_{i+1}) = S_{i+1}(x_{i+1})
     $$
   - **Непрерывность первой производной**: Первая производная сплайнов должна быть непрерывной:
     $$
     S_i'(x_{i+1}) = S_{i+1}'(x_{i+1})
     $$
   - **Непрерывность второй производной**: Вторая производная также должна быть непрерывной:
     $$
     S_i''(x_{i+1}) = S_{i+1}''(x_{i+1})
     $$

3. **Граничные условия**:
   Граничные условия определяют поведение сплайна на границах интервала. Существуют разные типы граничных условий:
   - **Неподвижные границы (natural spline)**: Вторая производная на границах равна нулю:
     $$
     S_0''(x_0) = 0 \quad \text{и} \quad S_{n-1}''(x_n) = 0
     $$
   - **Заданные производные**: Значения первой производной в крайних точках фиксируются

4. **Система уравнений**:
   Чтобы найти коэффициенты кубических сплайнов, необходимо решить систему линейных уравнений, которая формируется на основе условий непрерывности и производных. Это приводит к созданию матрицы, которую можно решить с помощью методов (численной) линейной алгебры

- **Преимущества**:
  - Высокая гладкость и непрерывность
  - Гибкость в подгонке под данные
  - Возможность работать с ограниченными наборами данных

- **Недостатки**:
  - Может быть сложным для реализации, особенно при большом количестве узлов
  - Появление осцилляций между узлами (особенно при использовании более высоких порядков)

  **Задание 9 (2 балла):** Реализуйте класс `CubicSplice` с заданными производными на границах. Чтобы лучше узнать про реализацию, посмотрите [тут](https://drive.google.com/file/d/1zQ_O-dqgZQoR4GmtAMvDLxkXEk4r2TU5/view?usp=share_link) в соответствующем разделе. Сравните с методами выше.

In [ ]:
class CubicSplice:
    """
    Notes
    -----
    This implementation does not use loops. Instead, it utilizes fully vectorized operations.
    """

    def __init__(
        self,
        nodes: np.ndarray,
        values: np.ndarray,
        left_first_der: float,
        right_first_der: float,
    ) -> None:
        """
        Initialize the cubic spline interpolator with boundary conditions.

        Parameters
        ----------
        nodes : np.ndarray
            The x-coordinates of the interpolation nodes.
        values : np.ndarray
            The y-coordinates corresponding to the nodes.
        left_first_der : float
            The first derivative at the left boundary.
        right_first_der : float
            The first derivative at the right boundary.
        """
        self.nodes = nodes
        self.values = values
        self.left_first_derivative = left_first_der
        self.right_first_derivative = right_first_der

    def locate_argument(self, x: np.ndarray) -> np.ndarray:
        """
        Locate the index of the interval to which each x belongs.

        Parameters
        ----------
        x : np.ndarray
            Array of x values to locate in the spline intervals.

        Returns
        -------
        np.ndarray
            Array of interval indices for each x.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    @staticmethod
    def compute_linear_function(
        x: np.ndarray, x0: float, y0: float, first_derivative: float
    ) -> np.ndarray:
        """
        Compute the value of the linear function given by a point and slope.

        Parameters
        ----------
        x : np.ndarray
            Array of x values to evaluate.
        x0 : float
            x-coordinate of the point.
        y0 : float
            y-coordinate of the point.
        first_derivative : float
            First derivative at the point.

        Returns
        -------
        np.ndarray
            Array of linear function values at x.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    @staticmethod
    def compute_cubic_spline_value(
        x_prev: float,
        x_curr: float,
        second_derivative_prev: float,
        second_derivative_curr: float,
        y_prev: float,
        y_curr: float,
        x: float,
    ) -> float:
        """
        Compute the value of the cubic spline in an interval.

        Parameters
        ----------
        x_prev : float
            Left node of the interval.
        x_curr : float
            Right node of the interval.
        second_derivative_prev : float
            Second derivative at the left node.
        second_derivative_curr : float
            Second derivative at the right node.
        y_prev : float
            Value of the function at the left node.
        y_curr : float
            Value of the function at the right node.
        x : float
            Point at which to evaluate the spline.

        Returns
        -------
        float
            The value of the cubic spline at x.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    def compute_system_matrix(self) -> np.ndarray:
        """
        Compute the system matrix for the cubic spline.

        Returns
        -------
        np.ndarray
            The tridiagonal matrix for the cubic spline system.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    def compute_right_part(self) -> np.ndarray:
        """
        Compute the right-hand side vector for the cubic spline system.

        Returns
        -------
        np.ndarray
            The right-hand side vector for the system.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    def compute_second_deriv_vector(self) -> np.ndarray:
        """
        Solve for the second derivatives of the cubic spline.

        Returns
        -------
        np.ndarray
            The second derivatives at each node.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict the interpolated values at x using the cubic spline.

        Parameters
        ----------
        x : np.ndarray
            Array of x values to interpolate.

        Returns
        -------
        np.ndarray
            Interpolated values at x.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

In [ ]:
# Your code here (∩ᄑ_ᄑ)⊃━☆ﾟ*･｡*

## Многомерная интерполяция

Функции, как известно, могут быть не только одномерными, но многомерными. Для таких функций уже не подойдут классические методы полиномами. Узнаем же, как интерполировать такие функции, на примере двухмерных

**Задание 10 (3 балла)**:

Рассмотрим функцию двух переменных, заданную на единичном квадрате: $[0; 1] \times [0; 1]$
$$
f(x_0, x_1) = \cos(2 \pi x_0) - \sin(2 \pi x_1) + \sinh(x_0 - x_1^2)
$$
Выполните следующие задачи:
1) Реализуйте алгоритм билинейной интерполяции, который по значениям функции, заданных сетке размера $n_0 \times n_1$ аппроксимирует значение исходной функции $f(x)$
2) Реализуйте алгоритм интерполяции на основе радиальных базисных функций.
3) Сравните точность работы алгоритмов. Для постройте интерполяционные функции для каждого из методов, используя значения функции, вычисленные на сетке.
4) Сравните точность алгоритма интерполяции при помощи радиальных базисных функций для двух сценариев: генерация узлов интерполяции при помощи сетки и случайная генерация узлов интерполяции.
5) Постройте график $log_2(r)$ vs $log_2(N)$, где $r$ - ошибка интерполяции, $N$ - число узлов интерполяции. В качестве ошибки можно использовать норму $\ell_{\infty}$:
$$
r = \max_{i \in \text{samples}}|y_{\text{pred}, i} - y_{\text{true}, i}|
$$
$N$ можно вычислить по размеру сетки в задаче билинейной интерполяции:
$N = n_0 n_1$. Для построения графика можно использовать следующие значения $n_0, n_1$:
$n_0 = n_1 = 10, 20, 40, 80, 160, 320, 640, ...$

### Интерполяция при помощи радиальных базисных функций.
Пусть $x_i$ $i=1, ... , N$ - узлы интерполяции в которых известны значения функции $f$: $y_i = f(x_i)$.

Можно вычислить параметр масштаба:
$$
s = \frac{s_1}{N \cdot d}
$$
Здесь $d$ - размерность пространства $x$, $N$ - число сэмплов, $s_1$ - константа.
В данной задаче $d=2$, $s_1 = 10$ - дает неплохие результаты с точки зрения интерполяции.

Функция интерполяции вычисляется следующим образом:
$$
f(x) \approx \sum_{i=1}^{N} w_i(x) y_i
$$
Здесь $w_i(x)$ - весовые функции, которые вычисляются следующим образом (аналогия softmax):
$$
w_i(x) = \frac{\exp(-|x - x_i|^2 / s^2)}{\sum_{j=1}^{N}\exp(-|x - x_j|^2 / s^2)}
$$

В общем случае, нет ограничений на точки $x_i$. Их можно генерировать случайным образом.

### Билинейная интерполяция.
В простейшем случае билинейной интерполяции можно предположить, что задана сетка точек на плоскости $x_{ij}$, $i=0, ..., n_0 - 1$, $j=0, ..., n_1 - 1$. Координаты точки $x_{ij}$ вычисляются следующим образом:
$$
x_{ij,0} = i / (n_0 - 1)
$$
$$
x_{ij,1} = j / (n_1 - 1)
$$
Помимо узлов интерполяция (сетки) заданы значения интерполируем ой функции в узлах интерполяции
Для произвольной точки $x$ находится прямоугольник вида $[x_{ij, 0}; x_{(i + 1)j, 0}] \times [x_{ij, 1}; x_{i(j + 1), 1}]$, которому она принадлежит.

Далее, вычисляются веса:
$$
W_0 = \frac{x_{(i + 1)j, 0} - x_0}{x_{(i + 1)j, 0} - x_{ij, 0}}
$$
$$
W_1 = \frac{x_{i(j + 1), 1} - x_1}{x_{i(j + 1), 1} - x_{ij, 1}}
$$
Значение интерполяционной функции вычисляется следующим образом:
$$
f(x) \approx W_0 W_1 y_{ij} + (1 - W_0) W_1 y_{(i + 1)j} + W_0 (1 - W_1) y_{i(j + 1)} + (1 - W_0) (1 - W_1) y_{(i + 1)(j + 1)}
$$


In [ ]:
class BilinearInterpolation:
    def __init__(self, nodes: np.ndarray, values: np.ndarray) -> None:
        """
        Initialize the Bilinear Interpolation model on a rectilinear 2D grid.

        Parameters
        ----------
        nodes : np.ndarray
            Grid node coordinates. Expected shape is (n0, n1, 2), where
            nodes[i, j] = [x0_ij, x1_ij] is the coordinate of the (i, j)-th grid node.
            The grid is assumed to be rectilinear and ordered along both axes.
        values : np.ndarray
            Function values at the grid nodes. Expected shape is (n0, n1), where
            values[i, j] = f(nodes[i, j]).
        """
        self.nodes = nodes.copy()
        self.values = values.copy()

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict interpolated values at x using bilinear interpolation.

        Parameters
        ----------
        x : np.ndarray
            Query points at which to interpolate. Expected shape is (m, 2),
            where each row is a point [x0, x1] in the unit square.

        Returns
        -------
        np.ndarray
            Interpolated values at the query points. Shape is (m,).

        Notes
        -----
        This implementation does not use loops. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    def bilinear_compute_weights(self, x: np.ndarray) -> np.ndarray:
        """
        Compute bilinear interpolation weights for each query point.

        Parameters
        ----------
        x : np.ndarray
            Query points. Expected shape is (m, 2).

        Returns
        -------
        np.ndarray
            Bilinear weights for each query point. Expected shape is (m, 4),
            corresponding to the four corners of the enclosing cell.

        Notes
        -----
        This implementation does not use loops. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

In [ ]:
class RadialBasisFunctionInterpolation:
    def __init__(self, nodes: np.ndarray, values: np.ndarray, dim: int, s1: int, num_samples: int) -> None:
        """
        Initialize the Radial Basis Function (RBF) interpolator with normalized Gaussian weights.

        Parameters
        ----------
        nodes : np.ndarray
            Interpolation nodes x_i. Expected shape is (N, dim), where N is the number
            of samples/nodes and `dim` is the input dimension.
        values : np.ndarray
            Function values at the nodes. Expected shape is (N,), where
            values[i] = f(nodes[i]).
        dim : int
            Dimensionality of the input space (for this task dim = 2).
        s1 : int
            Scale constant used to compute the kernel width.
        num_samples : int
            Number of interpolation nodes N. Used to compute the kernel width.
        """
        self.nodes = nodes.copy()
        self.values = values.copy()
        self.dim = dim
        self.s1 = s1
        self.num_samples = num_samples
        self.s = self.s1 / (self.num_samples * self.dim)

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predict interpolated values at x using normalized Gaussian RBF weights.

        Parameters
        ----------
        x : np.ndarray
            Query points at which to interpolate. Expected shape is (m, dim).

        Returns
        -------
        np.ndarray
            Interpolated values at the query points. Shape is (m,).

        Notes
        -----
        This implementation does not use loops. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

    def radial_basis_compute_weight(self, x: np.ndarray, node_i: np.ndarray, denominator: np.ndarray) -> np.ndarray:
        """
        Compute the normalized RBF weight w_i(x) for a single node.

        Parameters
        ----------
        x : np.ndarray
            Query points. Expected shape is (m, dim).
        node_i : np.ndarray
            A single interpolation node x_i. Expected shape is (dim,).
        denominator : np.ndarray
            The normalization term for each query point. Expected shape is (m,).

        Returns
        -------
        np.ndarray
            The normalized weights w_i(x) for the provided node across all query points.
            Shape is (m,).

        Notes
        -----
        This implementation does not use loops. Instead, it utilizes fully vectorized operations.
        """
        # Your code here (つ◕౪◕)つ━☆
        raise NotImplementedError

In [ ]:
# Your code here (つ◕౪◕)つ━☆

## Интерполяция изображений

Интерполировать можно не только функции, но и изображения (что тоже можно считать функцией, в целом).

Пусть у нас есть исходное изображение $I$ размером $W_{in} \times H_{in}$ и целевое $J$ размером $W_{out}\times H_{out}$

Для каждого пикселя $(u,v)$ целевого изображения (где $u,v$ — целые числа) мы находим его прообраз $(x,y)$ на исходном изображении (где $x,y$ — вещественные числа):

$$
s_x=\frac{W_{in}}{W_{out}},\qquad s_y=\frac{H_{in}}{H_{out}}
$$

$$
x=(u+0.5)s_x-0.5,\qquad y=(v+0.5)s_y-0.5
$$

- Метод ближайшего соседа (Nearest Neighbor):

    Округляем вещественные координаты до ближайшего целого: $$i=⌊x+0.5⌋ \quad j=⌊y+0.5⌋$$

    Тогда $I(u,v) = J(j,i)$

- Билинейная интерполяция (Bilinear):

    Линейная интерполяция по $x$ и $y$ на окрестности $2\times 2$.

    1. Базовый узел и дробные части:
    $$
    x_0=\lfloor x\rfloor,\qquad y_0=\lfloor y\rfloor
    $$
    $$
    \Delta x=x-x_0,\qquad \Delta y=y-y_0
    $$
    где $\Delta x,\Delta y\in[0,1)$.

    2. Четыре соседа:
    $$
    I_{00}=I(y_0,x_0),\
    I_{10}=I(y_0,x_0+1),
    I_{01}=I(y_0+1,x_0),
    I_{11}=I(y_0+1,x_0+1)
    $$
    (индексы берутся с обработкой границ)

    3. Взвешенная сумма:
    $$
    J(u,v)=(1-\Delta x)(1-\Delta y)I_{00}
    +\Delta x(1-\Delta y)I_{10}
    +(1-\Delta x)\Delta y,I_{01}
    +\Delta x\Delta y,I_{11}
    $$
        
- Бикубическая интерполяция:

    Сепарабельная кубическая свёртка по окрестности $4\times 4$.

    1. База и дробные части:
    $$
    x_0=\lfloor x\rfloor,\qquad y_0=\lfloor y\rfloor
    $$
    $$
    \Delta x=x-x_0,\qquad \Delta y=y-y_0
    $$

    2. Кубическое ядро с параметром $a=-0.75$:
    Пусть $t = |d|$. Тогда:
    $$
    w(d) = \begin{cases}
    (a+2)t^3-(a+3)t^2+1, & 0\le t<1\
    a t^3-5a t^2+8a t-4a, & 1\le t<2\
    0, & t\ge 2
    \end{cases}
    $$

    3. Суммирование по окрестности $4\times4$ (сепарабельность):
    $$
    J(u,v)=\sum_{n=-1}^{2}\sum_{m=-1}^{2}
    I(y_0+n,\ x_0+m)\cdot w(\Delta x-m)\cdot w(\Delta y-n)
    $$

**Задание 11 (2 балла):**

Реализуйте метод `resize` с разными методами интерполяции пикселей и сравните их между собой по качеству и скорости. Поэкспериментируйте с размерами, чтобы была видна визуальная разница между методами.

В качестве картинки возьмите вашу любую фотографию с Нового года. За смешные картинки получите бонус 0.1 балл

In [24]:
def pil_to_array(image: Image.Image) -> np.ndarray:
    to_tensor = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
    image_array = to_tensor(image)
    return image_array.numpy()

def array_to_pil(image: np.ndarray) -> Image.Image:
    to_pil = v2.ToPILImage()
    image_tensor = torch.from_numpy(image)
    image_pil = to_pil(image_tensor)
    return image_pil

def torch_resize(image: np.ndarray, out_size: tuple[int, int], mode: Literal['bilinear', 'bicubic']) -> np.ndarray:
    x = torch.from_numpy(image)
    x = x.unsqueeze(0)  # CHW -> NCHW
    if mode in ("bilinear", "bicubic"):
        y = F.interpolate(x, size=(out_h, out_w), mode=mode, align_corners=False)
    else:
        y = F.interpolate(x, size=(out_h, out_w), mode=mode)
    return y.squeeze(0).numpy()

#